<a href="https://colab.research.google.com/github/marco-siino/eloquent2024/blob/main/ELOQUENT_2024_Task_2_Hallucigen_Mistral7B_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [1]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets
!pip install -U deep-translator

import datasets
from datasets import load_dataset
from deep_translator import GoogleTranslator
import json
import re
import random
import numpy as np
import tqdm.notebook as tqdm

# Seed to shuffle the json training set.
seed_value = 42
random.seed(seed_value)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 191.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 264.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 130.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 164.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.74-cp310-cp310-linux_x86_64.whl size=61855856 sha256=7c49d3d4a20bb7c1096a5c9eadb816a051471fba20f94b86656614e195efb37e
  Stored in directory: /tmp/pip-ephem-wheel-cache-v4scoyns/wheels/21/1f/6d/ee16805ea65ed00d89776a8ce82f3631340a43fc41edabed97
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
 

Downloading an instruction-finetuned Mistral model.

In [2]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4, #16, # CPU cores
    n_batch=800, #8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loade

# Download the dataset for the three subtasks.

In [8]:
#load the trial data for both English and Swedish
trial_ds = load_dataset("Eloquent/HalluciGen-PG", name="trial")

#load the trial data only for Swedish
trial_ds_sv = load_dataset("Eloquent/HalluciGen-PG", name="trial", split="trial_swedish")

#load the test data for the detection step in both English and Swedish
test_ds = load_dataset("Eloquent/HalluciGen-PG", name="test_detection")

In [15]:
test_ds['test_detection_swedish'][0]

{'id': 0,
 'source': 'Kvinnor kommer att möta högre bilförsäkringspremier.',
 'hyp1': 'Det betyder att kvinnor kan förvänta sig att betala högre priser för sina fordonstilläggsförsäkringar.',
 'hyp2': 'Kvinnor kommer att få högre premier för bilförsäkring.'}

In [18]:
trial_ds['trial_english'][0]

{'id': 0,
 'source': 'The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.',
 'type': 'antonym',
 'hyp1': "In the majority of Sweden's 280 municipalities, the population has gone up.",
 'hyp2': "In the majority of Sweden's 280 municipalities, the population has gone down.",
 'label': 'hyp1'}

In [26]:
# Convert the dataset object to a list
trial_en_list = list(trial_ds['trial_english'])
trial_sv_list = list(trial_ds['trial_swedish'])

# Shuffle the list
random.shuffle(trial_en_list)
random.shuffle(trial_sv_list)



In [27]:
trial_sv_list

[{'id': 2,
  'source': 'Län med befolkningsminskning kommer att vara Vermillion, Posey och Madison.',
  'type': 'named entity',
  'hyp1': 'Vermillion, Posey och Madison är län som kommer att uppleva minskande befolkning.',
  'hyp2': 'Vermillion, Posey och Marion är län som kommer att uppleva minskande befolkning.',
  'label': 'hyp2'},
 {'id': 12,
  'source': 'Israels Peres uppmanar parterna att återgå till fredssamtalen.',
  'type': 'addition',
  'hyp1': 'Peres i Israel ger en uppmaning att återgå till fredssamtalen.',
  'hyp2': 'Israels president Shimon Peres uppmanade på måndagen världssamfundet att återuppta fredsförhandlingarna med palestinierna och betonade vikten av en tvåstatslösning.',
  'label': 'hyp2'},
 {'id': 11,
  'source': 'Grekisk högerextrem ledare fängslad i väntan på rättegång.',
  'type': 'addition',
  'hyp1': '\xa0I väntan på rättegång har en grekisk högerextrem ledare fängslats.',
  'hyp2': 'En grekisk högerextremistisk ledare har gripits och kommer att ställas inf

# Create few-shot samples from training set.

In [47]:
 prompt_context = """[INST] Which one of hyp1 and hyp2 is not supported by src?

"""

In [56]:
def create_few_shot_samples(train_list,nr_samples,language):

  nr_few_shot_samples = nr_samples

  few_shot_samples = ''

  for i in range(0,nr_few_shot_samples):
    if language=='sv':
      src = GoogleTranslator(source='sv', target='en').translate(train_list[i]['source'])
      hyp1 = GoogleTranslator(source='sv', target='en').translate(train_list[i]['hyp1'])
      hyp2 = GoogleTranslator(source='sv', target='en').translate(train_list[i]['hyp2'])
    else:
      src = train_list[i]['source']
      hyp1 = train_list[i]['hyp1']
      hyp2 = train_list[i]['hyp2']

    label = train_list[i]['label']

    few_shot_samples += '<s>'+prompt_context+"src: "+src+"\nhyp1: "+hyp1 + '\nhyp2: '+hyp2+'\nlabel: [/INST] \n ' + label + "\n</s> \n\n "

  return few_shot_samples

In [63]:
few_shot_samples_en = create_few_shot_samples(trial_en_list,16,'en')

In [65]:
few_shot_samples_sv = create_few_shot_samples(trial_sv_list,20,'sv')

In [66]:
print(few_shot_samples_en)

<s>[INST] Which one of hyp1 and hyp2 is not supported by src?

src: This state of affairs has not changed in more than 100 years, but hopefully at some stage - and perhaps soon - change will come.
hyp1: There has been no change in the status quo in over 100 years, but there is hope that change will soon come. 
hyp2: The state of affairs is1-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-6556
label: [/INST] 
 hyp2
</s> 

 <s>[INST] Which one of hyp1 and hyp2 is not supported by src?

src: The draft agenda as drawn up by the Conference of Presidents pursuant to Rule 95 of the Rules of Procedure has been distributed.
hyp1: The Conference of Presidents hasn't distributed the draft agenda.
hyp2: The Conference of Presidents has di

In [67]:
print(few_shot_samples_sv)

<s>[INST] Which one of hyp1 and hyp2 is not supported by src?

src: Counties with population declines will be Vermillion, Posey and Madison.
hyp1: Vermillion, Posey and Madison are counties that will experience declining populations.
hyp2: Vermillion, Posey and Marion are counties that will experience declining populations.
label: [/INST] 
 hyp2
</s> 

 <s>[INST] Which one of hyp1 and hyp2 is not supported by src?

src: Israel's Peres calls on the parties to return to peace talks.
hyp1: Peres in Israel calls for return to peace talks.
hyp2: Israeli President Shimon Peres on Monday called on the world community to resume peace talks with the Palestinians, stressing the importance of a two-state solution.
label: [/INST] 
 hyp2
</s> 

 <s>[INST] Which one of hyp1 and hyp2 is not supported by src?

src: Greek far-right leader jailed awaiting trial.
hyp1: Pending trial, a Greek far-right leader has been imprisoned.
hyp2: A Greek far-right leader has been arrested and will stand trial, accus

# Run! (English test set)

In [72]:
#replies_list = ['YES','NO']
counter = 0

# List to store the results.
prediction_en = []

for i in range(0,len(test_ds['test_detection_english'])):
  src = test_ds['test_detection_english'][i]['source']
  hyp1 = test_ds['test_detection_english'][i]['hyp1']
  hyp2 = test_ds['test_detection_english'][i]['hyp2']
  id = test_ds['test_detection_english'][i]['id']

  current_sample = prompt_context+"src: "+src+"\nhyp1: "+hyp1 + '\nhyp2: '+hyp2+'\nlabel: [/INST] \n '
  prompt = few_shot_samples_en+current_sample

  #print(prompt)
  #print(current_sample)

  response = lcpp_llm(
        prompt=prompt,
        temperature= 0.2,
        logprobs=1,
        #max_tokens =1
      )

  #print(response)

  answer = str(response["choices"][0]["text"]).strip()
  #print(answer)
  answer = answer[:4]
  #answer = answer.split()[0]
  # Sometime output contains a '.' remove it!
  #answer = answer.replace('.','')

  # If the predicted word is not in emotion list just replace with neutral.
  #if answer not in replies_list:

  #current_sample += answer + " \n "

  print("GENERATED: "+ current_sample+'\n'+answer)

  current_element = {
        "id": id,
        "label": answer,
        "explanation": ""
    }
  prediction_en.append(current_element)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       7.89 ms /    16 runs   (    0.49 ms per token,  2028.65 tokens per second)
llama_print_timings: prompt eval time =   71482.99 ms /    85 tokens (  840.98 ms per token,     1.19 tokens per second)
llama_print_timings:        eval time =    1574.25 ms /    15 runs   (  104.95 ms per token,     9.53 tokens per second)
llama_print_timings:       total time =    2264.06 ms /   100 tokens


GENERATED: [INST] Which one of hyp1 and hyp2 is not supported by src?

src: It has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout Europe.
hyp1: You can support and encourage an exchange of experiences to raise the level of competence in Europe.
hyp2: We can support and encourage an exchange of experiences to raise the level of competence in Europe.
label: [/INST] 
 
hyp1


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       7.60 ms /    16 runs   (    0.48 ms per token,  2104.43 tokens per second)
llama_print_timings: prompt eval time =     305.73 ms /    95 tokens (    3.22 ms per token,   310.73 tokens per second)
llama_print_timings:        eval time =    1645.82 ms /    15 runs   (  109.72 ms per token,     9.11 tokens per second)
llama_print_timings:       total time =    2282.81 ms /   110 tokens


GENERATED: [INST] Which one of hyp1 and hyp2 is not supported by src?

src: Therefore, I am calling for an increase in the premiums for all varieties of leaf tobacco for the 1999, 2000 and 2001 harvests.
hyp1: I want to see a decrease in the premiums for all varieties of leaf tobacco.
hyp2: I want to see an increase in the premiums for all varieties of leaf tobacco.
label: [/INST] 
 
hyp1


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       9.00 ms /    16 runs   (    0.56 ms per token,  1777.38 tokens per second)
llama_print_timings: prompt eval time =     304.41 ms /   105 tokens (    2.90 ms per token,   344.93 tokens per second)
llama_print_timings:        eval time =    1756.98 ms /    15 runs   (  117.13 ms per token,     8.54 tokens per second)
llama_print_timings:       total time =    2457.46 ms /   120 tokens


...

GENERATED: [INST] Which one of hyp1 and hyp2 is not supported by src?

src: Mr President, let me express my thanks for the preceding remarks and extend a special welcome to my honourable colleague the Transport Commissioner.
hyp1: Mr President, I would like to extend a warm welcome to the Transport Commissioner.
hyp2: Madam President, I would like to extend a warm welcome to the Transport Commissioner.
label: [/INST] 
 
hyp1


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       9.40 ms /    16 runs   (    0.59 ms per token,  1701.40 tokens per second)
llama_print_timings: prompt eval time =     297.01 ms /   108 tokens (    2.75 ms per token,   363.63 tokens per second)
llama_print_timings:        eval time =    1858.07 ms /    15 runs   (  123.87 ms per token,     8.07 tokens per second)
llama_print_timings:       total time =    2510.38 ms /   123 tokens


GENERATED: [INST] Which one of hyp1 and hyp2 is not supported by src?

src: We can replace 16 out of 626 Members of the European Parliament, but we can never again as voters decide to bring in a new law.
hyp1: As voters we cannot decide on a new law, but we can replace 16 out of 626 Members of the European Parliament.
hyp2: As voters decide on a new law, we can replace 16 out of 626 Members of the European Parliament.
label: [/INST] 
 
hyp2


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       7.44 ms /    16 runs   (    0.46 ms per token,  2151.41 tokens per second)
llama_print_timings: prompt eval time =     292.96 ms /    88 tokens (    3.33 ms per token,   300.38 tokens per second)
llama_print_timings:        eval time =    1681.97 ms /    15 runs   (  112.13 ms per token,     8.92 tokens per second)
llama_print_timings:       total time =    2245.92 ms /   103 tokens


GENERATED: [INST] Which one of hyp1 and hyp2 is not supported by src?

src: The Council is making a mistake by leaving it entirely up to the United States to formulate proposals for a solution to the Middle East problem.
hyp1: The United States should not be the only ones to come up with a solution to the Middle East problem
hyp2: The United States needs to come up with a solution to the Middle East problem, not the Council.
label: [/INST] 
 
hyp2


In [73]:
print(prediction_en)

[{'id': 0, 'label': 'hyp1', 'explanation': ''}, {'id': 1, 'label': 'hyp1', 'explanation': ''}, {'id': 2, 'label': 'hyp1', 'explanation': ''}, {'id': 3, 'label': 'hyp1', 'explanation': ''}, {'id': 4, 'label': 'hyp2', 'explanation': ''}, {'id': 5, 'label': 'hyp1', 'explanation': ''}, {'id': 6, 'label': 'hyp1', 'explanation': ''}, {'id': 7, 'label': 'hyp1', 'explanation': ''}, {'id': 8, 'label': 'hyp1', 'explanation': ''}, {'id': 9, 'label': 'hyp2', 'explanation': ''}, {'id': 10, 'label': 'hyp1', 'explanation': ''}, {'id': 11, 'label': 'hyp1', 'explanation': ''}, {'id': 12, 'label': 'hyp1', 'explanation': ''}, {'id': 13, 'label': 'hyp1', 'explanation': ''}, {'id': 14, 'label': 'hyp2', 'explanation': ''}, {'id': 15, 'label': 'hyp1', 'explanation': ''}, {'id': 16, 'label': 'hyp1', 'explanation': ''}, {'id': 17, 'label': 'hyp1', 'explanation': ''}, {'id': 18, 'label': 'hyp1', 'explanation': ''}, {'id': 19, 'label': 'hyp2', 'explanation': ''}, {'id': 20, 'label': 'hyp1', 'explanation': ''}, {

In [ ]:
# List to store the results.
prediction_sv = []

for i in range(0,len(test_ds['test_detection_swedish'])):
  src = GoogleTranslator(source='sv', target='en').translate(test_ds['test_detection_swedish'][i]['source'])
  hyp1 = GoogleTranslator(source='sv', target='en').translate(test_ds['test_detection_swedish'][i]['hyp1'])
  hyp2 = GoogleTranslator(source='sv', target='en').translate(test_ds['test_detection_swedish'][i]['hyp2'])
  id = test_ds['test_detection_swedish'][i]['id']

  current_sample = prompt_context+"src: "+src+"\nhyp1: "+hyp1 + '\nhyp2: '+hyp2+'\nlabel: [/INST] \n '
  prompt = few_shot_samples_sv+current_sample

  #print(prompt)
  #print(current_sample)

  response = lcpp_llm(
        prompt=prompt,
        temperature= 0.2,
        logprobs=1,
        #max_tokens =1
      )

  #print(response)

  answer = str(response["choices"][0]["text"]).strip()
  #print(answer)
  answer = answer[:4]
  #answer = answer.split()[0]
  # Sometime output contains a '.' remove it!
  #answer = answer.replace('.','')

  # If the predicted word is not in emotion list just replace with neutral.
  #if answer not in replies_list:

  #current_sample += answer + " \n "

  print("GENERATED: "+ current_sample+'\n'+answer)

  current_element = {
        "id": id,
        "label": answer,
        "explanation": ""
    }
  prediction_sv.append(current_element)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       8.08 ms /    16 runs   (    0.50 ms per token,  1980.69 tokens per second)
llama_print_timings: prompt eval time =    4018.19 ms /  2593 tokens (    1.55 ms per token,   645.32 tokens per second)
llama_print_timings:        eval time =    1519.75 ms /    15 runs   (  101.32 ms per token,     9.87 tokens per second)
llama_print_timings:       total time =   13216.20 ms /  2608 tokens


GENERATED: [INST] Which one of hyp1 and hyp2 is not supported by src?

src: Women will face higher car insurance premiums.
hyp1: This means women can expect to pay higher prices for their vehicle supplement insurance.
hyp2: Women will receive higher car insurance premiums.
label: [/INST] 
 
hyp2


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       7.67 ms /    16 runs   (    0.48 ms per token,  2086.59 tokens per second)
llama_print_timings: prompt eval time =     323.04 ms /   104 tokens (    3.11 ms per token,   321.94 tokens per second)
llama_print_timings:        eval time =    1551.31 ms /    15 runs   (  103.42 ms per token,     9.67 tokens per second)
llama_print_timings:       total time =    2173.91 ms /   119 tokens


GENERATED: [INST] Which one of hyp1 and hyp2 is not supported by src?

src: Operating income was $1.45 billion, up from last year's earnings of $1.38 billion.
hyp1: Revenue from the operation was $1.45 billion, down from the previous year's result of $1.38 billion.
hyp2: Revenue from the operation was $1.45 billion, up from the previous year's result of $1.38 billion.
label: [/INST] 
 
hyp1


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       7.93 ms /    16 runs   (    0.50 ms per token,  2017.15 tokens per second)
llama_print_timings: prompt eval time =     280.02 ms /    60 tokens (    4.67 ms per token,   214.27 tokens per second)
llama_print_timings:        eval time =    1587.35 ms /    15 runs   (  105.82 ms per token,     9.45 tokens per second)
llama_print_timings:       total time =    2056.53 ms /    75 tokens


GENERATED: [INST] Which one of hyp1 and hyp2 is not supported by src?

src: Mandela back in hospital in 'serious but stable' condition.
hyp1: Mandela does not return to hospital in serious, but stable, health.
hyp2: Mandela returns to hospital in serious but stable condition.
label: [/INST] 
 
hyp1


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1301.19 ms
llama_print_timings:      sample time =       8.46 ms /    16 runs   (    0.53 ms per token,  1891.48 tokens per second)
llama_print_timings: prompt eval time =     327.76 ms /   114 tokens (    2.88 ms per token,   347.81 tokens per second)
llama_print_timings:        eval time =    1552.58 ms /    15 runs   (  103.51 ms per token,     9.66 tokens per second)
llama_print_timings:       total time =    2235.44 ms /   129 tokens


# Function to create the CSV files containing the predictions.

In [ ]:
import csv

def create_csv_prediction_file (data, filename):

  # Data to be written to the CSV file
  #data = [
  #    {"id": 1, "label": "A", "explanation": "Explanation for A"},
  #    {"id": 2, "label": "B", "explanation": "Explanation for B"},
  #    {"id": 3, "label": "C", "explanation": "Explanation for C"},
  #]

  # Name of the CSV file
  #csv_file = "data.csv"
  csv_file = filename

  # Field names
  fields = ["id", "label", "explanation"]

  # Writing to CSV file
  with open(csv_file, mode='w', newline='') as file:
      writer = csv.DictWriter(file, fieldnames=fields)

      # Write the header
      writer.writeheader()

      # Write the data
      for row in data:
          writer.writerow(row)

  print("CSV file created successfully.")


In [ ]:
create_csv_prediction_file (prediction_en, 'eloquent2024_mc_mistral_en_prediction.csv')
create_csv_prediction_file (prediction_sv, 'eloquent2024_mc_mistral_sv_prediction.csv')

In [ ]:
from google.colab import files
files.download('eloquent2024_mc_mistral_en_prediction.csv')
files.download('eloquent2024_mc_mistral_sv_prediction.csv')